# Imports

In [1]:
import pandas as pd
import vectorbtpro as vbt
import numpy as np

# Initialization

In [2]:
vbt.settings.wrapping ["freq"]                = "1m"
vbt.settings.portfolio['init_cash']           = 10000

# Settings and Parameters

In [3]:
pickle_files_path = "../data/LSTM_pw75_lb250_bt2000_mem5000/*.pkl"
prediction_window = 75

excel_output_file_name  = f"../results/{pickle_files_path.split('/')[-2] + '.xlsx'}"

# Local Imports

In [4]:
import os
import sys

sys.path.append(os.getcwd())

In [5]:
from lstm_analysis_utils import (read_pickle_files_into_df, add_forward_prices_to_df, generate_fwd_actual_column
                                 , generate_df_with_euclidean_distances, calculate_slopes, calculate_correlation_slopes
                                )
from lstm_results_utils import (export_results, store_backtest_results)
from lstm_analysis_constants import (ActionType, EntryType, LSTM_REVERSAL_EXITS_BACKTEST_RESULT_KEY
                                     , LSTM_PREDICTION_WINDOW_EXITS_BACKTEST_RESULT_KEY)
from quantile_value import generate_quantile_bands
from prediction_window_slopes import PredictionWindowSlopes

# Processing

In [6]:
df = read_pickle_files_into_df(pickle_files_path)

In [7]:
add_forward_prices_to_df(df, prediction_window)
df = df.copy()  # for large prediction_window size, the copy() call eliminates the fragmented dataframe warning

In [8]:
generate_fwd_actual_column(df)

In [9]:
df = generate_df_with_euclidean_distances(df, prediction_window)

In [10]:
calculate_slopes(df)

In [11]:
calculate_correlation_slopes(df)

Correlation between Euclidean distance between long array and short array and future actual results:  0.20583292687468935
Correlation between difference in long minus short predictions and future actual results for longs:  0.7675232839485857
Correlation between difference in long minus short predictions and future actual results for shorts:  0.7698060560552524
Correlation between long slopes and future results: 0.02781757293641048
Correlation between short slopes and future results: -0.01740683522317173


# Running backtest

- ### Storing the results of the backtests

In [12]:
results_as_list  = []

- ### Baseline backtest - just listen to LSTM

In [13]:
# Exits are from reversals, as our LSTM model doesn't produce exit signals
entries         = pd.Series(np.where((df['recommendations'] == ActionType.OPEN_LONG  ), True, False))
exits           = pd.Series(np.where((df['recommendations'] == ActionType.CLOSE_LONG ), True, False))
short_entries   = pd.Series(np.where((df['recommendations'] == ActionType.OPEN_SHORT ), True, False))
short_exits     = pd.Series(np.where((df['recommendations'] == ActionType.CLOSE_SHORT), True, False))

pf = vbt.Portfolio.from_signals(
    high                = df['BTCUSDT_High'],
    low                 = df['BTCUSDT_Low'],
    open                = df['BTCUSDT_Open'],
    close               = df['BTCUSDT_Close'],
    entries             = entries, # commented out for a short only backtest
    exits               = exits,
    short_entries       = short_entries,
    short_exits         = short_exits,    
    time_delta_format   = 'Rows', # Use the row index to calculate the time delta    
    )

store_backtest_results(LSTM_REVERSAL_EXITS_BACKTEST_RESULT_KEY, pf, results_as_list, None, True)

--------LSTM_only_reversal_exits--------
Total Return [%]: 93.81
Win Rate [%]: 50.24
Total Trades: 205


In [14]:
# Exits are controlled by td_stop
entries         = pd.Series(np.where((df['recommendations'] == ActionType.OPEN_LONG  ), True, False))
short_entries   = pd.Series(np.where((df['recommendations'] == ActionType.OPEN_SHORT ), True, False))

pf = vbt.Portfolio.from_signals(
    high                = df['BTCUSDT_High'],
    low                 = df['BTCUSDT_Low'],
    open                = df['BTCUSDT_Open'],
    close               = df['BTCUSDT_Close'],
    entries             = entries, # commented out for a short only backtest    
    short_entries       = short_entries,     
    td_stop             = prediction_window,
    time_delta_format   = 'Rows', # Use the row index to calculate the time delta    
    )
store_backtest_results(LSTM_PREDICTION_WINDOW_EXITS_BACKTEST_RESULT_KEY, pf, results_as_list, None, True)

--------LSTM_only_prediction_window_exits--------
Total Return [%]: 13.49
Win Rate [%]: 51.35
Total Trades: 555


- ### Using different slope quantiles

In [15]:
num_quantiles               = 5
threshold_increment         = 0.0001
quantiles                   = np.linspace(0, 1, num=num_quantiles + 1)

- #### long_slope and short_slope

In [16]:
long_slope_quantile_bands  = generate_quantile_bands(df["long_slope"].quantile(quantiles))
short_slope_quantile_bands = generate_quantile_bands(df["short_slope"].quantile(quantiles))  

In [17]:
for long_band in long_slope_quantile_bands:
  for short_band in short_slope_quantile_bands:    
    entry_slope_threshold       = [x for x in np.arange(long_band.lower_bound.value , long_band.upper_bound.value , threshold_increment)]
    short_entry_slope_threshold = [x for x in np.arange(short_band.lower_bound.value, short_band.upper_bound.value, threshold_increment)]

    for t1 in entry_slope_threshold:    
      for t2 in short_entry_slope_threshold:        
        entries       = pd.Series(np.where((df['long_slope' ] > t1  ), True, False))
        short_entries = pd.Series(np.where((df['short_slope'] < -t2 ), True, False))        

        num_trades = (entries == True).sum() + (short_entries == True).sum()

        if num_trades > 0:    
          pf = vbt.Portfolio.from_signals(
              high              = df['BTCUSDT_High'],
              low               = df['BTCUSDT_Low'],
              open              = df['BTCUSDT_Open'],
              close             = df['BTCUSDT_Close'],
              entries           = entries, # commented out for a short only backtest          
              short_entries     = short_entries,
              td_stop           = prediction_window, # Hold on to the position for 8 bars
              time_delta_format = 'Rows', # Use the row index to calculate the time delta              
              accumulate        = False,
              # sl_stop = 0.005,
              )    
          
          key = f"Prediction window exits"
          slopes = PredictionWindowSlopes(t1, t2, None, EntryType.LONG_SHORT)
          store_backtest_results(key, pf, results_as_list, slopes)      

In [18]:
for long_band in long_slope_quantile_bands:
  for short_band in short_slope_quantile_bands:    
    entry_slope_threshold       = [x for x in np.arange(long_band.lower_bound.value , long_band.upper_bound.value , threshold_increment)]
    short_entry_slope_threshold = [x for x in np.arange(short_band.lower_bound.value, short_band.upper_bound.value, threshold_increment)]

    for t1 in entry_slope_threshold:      
      exit_t1_threshold = t1 * 0.5

      for t2 in short_entry_slope_threshold:
        exit_t2_threshold = t2 * 0.5

        entries       = pd.Series(np.where((df['long_slope' ] > t1  ), True, False))
        short_entries = pd.Series(np.where((df['short_slope'] < -t2 ), True, False))

        exits         = pd.Series(np.where((df['long_slope' ] < exit_t1_threshold  ), True, False)) 
        short_exits   = pd.Series(np.where((df['short_slope'] > -exit_t2_threshold ), True, False))

        num_trades = (entries == True).sum() + (short_entries == True).sum()

        if num_trades > 0:    
          pf = vbt.Portfolio.from_signals(
              high              = df['BTCUSDT_High'],
              low               = df['BTCUSDT_Low'],
              open              = df['BTCUSDT_Open'],
              close             = df['BTCUSDT_Close'],
              entries           = entries, # commented out for a short only backtest          
              exits             = exits,
              short_entries     = short_entries,              
              short_exits       = short_exits,
              time_delta_format = 'Rows', # Use the row index to calculate the time delta              
              accumulate        = False,
              # sl_stop = 0.005,
              )    
          
          key = f"Fractional slope exits"
          slopes = PredictionWindowSlopes(t1, t2, None, EntryType.LONG_SHORT)
          store_backtest_results(key, pf, results_as_list, slopes)      

- #### long_minus_short

In [19]:
long_minus_short_min = df['long_minus_short'].min()
long_minus_short_max = df['long_minus_short'].max()

long_minus_short_thresholds = [x for x in np.arange(long_minus_short_min, long_minus_short_max, threshold_increment)]

for threshold in long_minus_short_thresholds:
    entries = pd.Series(np.where((df['long_minus_short'] < threshold), True, False))

    num_trades = (entries == True).sum()

    if num_trades > 0:
        pf = vbt.Portfolio.from_signals(
            high              = df['BTCUSDT_High'],
            low               = df['BTCUSDT_Low'],
            open              = df['BTCUSDT_Open'],
            close             = df['BTCUSDT_Close'],
            entries           = entries, # commented out for a short only backtest    
            td_stop           = prediction_window, # Hold on to the position for 8 bars
            time_delta_format = 'Rows', # Use the row index to calculate the time delta    
            accumulate        = False,    
            )
        
        key = f"Long minus short"    
        slopes = PredictionWindowSlopes(None, None, threshold, EntryType.LONG_ONLY)
        store_backtest_results(key, pf, results_as_list, slopes)

In [27]:
for threshold in long_minus_short_thresholds:
    short_entries = pd.Series(np.where((df['long_minus_short'] < threshold), True, False))

    num_trades = (short_entries == True).sum()

    if num_trades > 0:
        pf = vbt.Portfolio.from_signals(
            high              = df['BTCUSDT_High'],
            low               = df['BTCUSDT_Low'],
            open              = df['BTCUSDT_Open'],
            close             = df['BTCUSDT_Close'],
            short_entries     = short_entries, # commented out for a short only backtest    
            td_stop           = prediction_window, # Hold on to the position for 8 bars
            time_delta_format = 'Rows', # Use the row index to calculate the time delta    
            accumulate        = False,    
            )
        
        key = f"Long minus short"    
        slopes = PredictionWindowSlopes(None, None, threshold, EntryType.SHORT_ONLY)
        store_backtest_results(key, pf, results_as_list, slopes)

# Exporting the results

In [28]:
results_df = export_results(results_as_list)
results_df.to_excel(excel_output_file_name)

- ### Other backtests - work in progress and code may not execute at all!!!

In [22]:
df['long_slope'].describe()

count   42718.00000
mean       -0.00008
std         0.00068
min        -0.00105
25%        -0.00074
50%        -0.00014
75%         0.00056
max         0.00105
Name: long_slope, dtype: float64

In [23]:
df['short_slope'].quantile(quantiles)

0.00000   -0.00105
0.20000   -0.00076
0.40000   -0.00027
0.60000    0.00027
0.80000    0.00075
1.00000    0.00105
Name: short_slope, dtype: float64

In [24]:
df['actual_slope'].describe()

count   42718.00000
mean       -0.00000
std         0.00001
min        -0.00002
25%        -0.00000
50%        -0.00000
75%         0.00000
max         0.00002
Name: actual_slope, dtype: float64

In [25]:
df['long_minus_short'].describe()

count   42718.00000
mean        0.87596
std         0.20377
min         0.10446
25%         0.72646
50%         0.87046
75%         1.02709
max         1.41042
Name: long_minus_short, dtype: float64

In [26]:
# Set up entries and exits based on the slope
entry_slope_threshold = 0.0000
short_entry_slope_threshold = 0.0000
entries = pd.Series(np.where((df['long_slope'] > entry_slope_threshold), True, False))
long_minus_short_t = 0.4
entries = pd.Series(np.where((df['long_minus_short'] < long_minus_short_t), True, False))
entries = pd.Series(np.where((df['long_minus_short'] < long_minus_short_t & long_slope > ??), True, False))
#short_entries = pd.Series(np.where((new_df['short_slope'] < -short_entry_slope_threshold), True, False))
#short_entries = pd.Series(np.where((new_df['long_minus_short'] < long_minus_short_t), True, False))
pf = vbt.Portfolio.from_signals(
    high=df['BTCUSDT_High'],
    low=df['BTCUSDT_Low'],
    open=df['BTCUSDT_Open'],
    close=df['BTCUSDT_Close'],
    entries=entries, # commented out for a short only backtest
    # exits = result_df['long_slope']<0.04,
    #short_entries=short_entries,
    td_stop = prediction_window, # Hold on to the position for 8 bars
    time_delta_format = 'Rows', # Use the row index to calculate the time delta
    # tp_stop = 0.01,
    accumulate=False,
    # sl_stop = 0.005,
    )
print(pf.stats())

SyntaxError: invalid syntax (497242019.py, line 7)

In [ ]:
# print(f"Num entries = {(entries == True).sum()}")
# print(f"Num short entries = {(short_entries == True).sum()}")
# new_df["long_slope"].describe()